# Entrenamiento de los agentes basados en redes neuronales

Para entrenar a los agentes, deberemos definir un set de datos con estados del juego y un set de etiquetas con la acción preferida (representada en un vector de largo 5 con ceros en todas las posiciones excepto la de la acción elegida).

Luego, entrenaremos la red neuronal en este set de datos y la guardaremos en un archivo (``NN[Agent].h5``), el cual podrá ser accedido por el agente en el archivo ``neural.py``.

In [16]:
# Librerías utilizadas para trabajar en el notebook
import tensorflow as tf # Instalar mediante pip install tensorflow
import numpy as np
import random
import os

In [17]:
# Funciones auxiliares (que pueden resultar útiles) y rival de baseline, entregado con la tarea
from agents.baseline import BaseCat, BaseMouse
from utils import bfs_search, get_valid_moves

In [18]:
# Cargamos el mapa de juego, para poder buscar espacios libres dentro de él
lab_map = np.load(os.path.join("game_map.npy"))

In [19]:
# Obtenemos los espacios libres dentro del mapa
free_positions = []

for x in range(lab_map.shape[0]):
    for y in range(lab_map.shape[1]):
        if lab_map[x, y] == 0:
            free_positions.append((x, y))

free_positions[:5]

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]

## Entrenamiento del gato 🐈

### Creación del set de datos

Generaremos un set de datos ``X`` que contendrá numpy arrays con la posición del gato y el ratón ``(cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1])``.

Además, generaremos un set de etiquetas ``y`` que contendrá numpy arrays con el movimiento a realizar (ej. ``(0, 0, 1, 0, 0)`` significa hacer el movimiento 2).

La forma en que calculas qué acción llevar a cabo dado el estado de juego es tu decisión, puede ser una búsqueda utilizando el algoritmo de BFS entregado, una política manual o una serie de reglas if-else. Lo importante es que este será el comportamiento que la red neuronal del agente tratará de aproximar. Recuerda que dispones del mapa de juego en ``lab_map`` y de un agente contra el que entrenar bajo la clase `BaseMouse`.

In [20]:
# Instanciamos al jugador oponente
rival = BaseMouse((0, 0))

# Trabajaremos a X e y como listas, en la siguiente celda las transformaremos a un array de numpy
# (los componentes que colocas en ellas si deben ser numpy arrays)
X = []
y = []

# Para cada combinación de posiciones de gato y ratón
for cat_pos in free_positions:
    for mouse_pos in free_positions:
        X.append(np.array([cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1]]))

        # ===== COMPLETAR =====
        # Se debe calcular un vector de accion, correspondiente a la decisión a tomar para añadirlo a la lista y
        label = np.zeros((5))   # Actualmente se trata de un vector de solo ceros, se debe reemplazar la posición del movimiento elegido por un 1
        # =====================

        # # Convertimos las posiciones a arrays de numpy
        cat_pos_np = np.array(cat_pos)
        mouse_pos_np = np.array(mouse_pos)

        # Obtenemos la acción que tomaría el gato y el ratón
        mouse_action = rival.get_action(lab_map, cat_pos_np, mouse_pos_np)

        # Representamos las acciones en formato one-hot
        label[mouse_action] = 1

        # Añadimos las etiquetas a la lista y
        y.append(label)

In [21]:
len(X), len(y)

(6561, 6561)

In [22]:
# Transformamos las listas a numpy arrays
X = np.array(X)
y = np.array(y)

### Diseño y entrenamiento de la Red Neuronal

Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

In [23]:
# Recuerda que para esta tarea la entrada es un vector de 4 elementos y buscamos una salida de 5 elementos (la probabilidad de jugar acción posible)

# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees
# model = tf.keras.Sequential([])
# =====================

# Creación del modelo de red neuronal
model = tf.keras.Sequential([
    # Capa de entrada
    tf.keras.Input(shape=(4,)),  # La entrada es un vector de 4 elementos

    # Capas ocultas
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(500, activation='relu'),
    tf.keras.layers.Dense(900, activation='relu'),
    tf.keras.layers.Dense(10),

    # Capa de salida
    tf.keras.layers.Dense(5, activation="softmax")  # La salida es un vector de 5 elementos (probabilidad para cada acción)
])


TensorFlow nos permite además compilar un modelo, especificando qué optimizador utilizar para su ajuste de parámetros y qué función de pérdida utilizar para hacer la backpropagation.

Compilaremos un modelo de la forma:
``model.compile(optimizer = 'optimizer_name', loss = 'loss_name', metrics = ['accuracy'])``

- Con `optimizer_name` el nombre de algún optimizador (por ejemplo, `sgd` o `adam`, más optimizadores en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))
- Con `loss_name` el nombre de alguna función de pérdida (por ejemplo, `binary_crossentropy`, más funciones en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/losses))
- Usaremos de métrica el accuracy del modelo, aunque existe [múltiples otras métricas que incluir](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [24]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow
# model
# =====================

# Compilación del modelo
model.compile(
    optimizer='adam',  # Optimizador
    loss='categorical_crossentropy',  # Función de pérdida
    metrics=['accuracy']  # Métricas para monitorear durante el entrenamiento
)


Ahora que especificamos cómo vamos calcular el error del modelo y cómo lo usaremos para ajustar sus pesos, es hora de entrenar nuestra red. Para ello, utilizaremos su método `.fit()` de la siguiente forma:

`model.fit(X, y, epochs = a, batch_size = b)`

- `X` corresponde a nuestra matriz de características
- `y` corresponde a nuestro vector de etiquetas para cada muestra
- `epochs` determina cuántas veces iteraremos sobre todo el set de datos
- `batch_size` determina cuántas muestras serán consideradas por propagación del gradiente (un valor mayor significa entrenamiento más rápido pero mayor uso de memoria)

In [36]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()
# model
# =====================

# Entrenamiento del modelo
history = model.fit(
    X,  # Matriz de características
    y,  # Vector de etiquetas
    epochs=30,  # Número de épocas
    batch_size=512  # Tamaño del lote
)


Epoch 1/30
13/13 [==============================] - 0s 26ms/step - loss: 0.1606 - accuracy: 0.9354
Epoch 2/30
13/13 [==============================] - 0s 25ms/step - loss: 0.1241 - accuracy: 0.9560
Epoch 3/30
13/13 [==============================] - 0s 26ms/step - loss: 0.1163 - accuracy: 0.9587
Epoch 4/30
13/13 [==============================] - 0s 25ms/step - loss: 0.0951 - accuracy: 0.9688
Epoch 5/30
13/13 [==============================] - 0s 25ms/step - loss: 0.0807 - accuracy: 0.9776
Epoch 6/30
13/13 [==============================] - 0s 27ms/step - loss: 0.0775 - accuracy: 0.9784
Epoch 7/30
13/13 [==============================] - 0s 29ms/step - loss: 0.0746 - accuracy: 0.9803
Epoch 8/30
13/13 [==============================] - 0s 31ms/step - loss: 0.0703 - accuracy: 0.9800
Epoch 9/30
13/13 [==============================] - 0s 32ms/step - loss: 0.0677 - accuracy: 0.9819
Epoch 10/30
13/13 [==============================] - 0s 33ms/step - loss: 0.0669 - accuracy: 0.9825
Epoch 11/

Por último, guardaremos el modelo entrenado en el directorio `agents/data/NNCat.h5` para poder acceder a este modelo desde nuestra clase `NNCat`:

In [37]:
model.save(os.path.join('agents', 'data', 'NNCat.h5'))

c:\Users\Rodrigo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Entrenamiento del ratón 🐁

### Creación del set de datos

Generaremos un set de datos ``X`` que contendrá numpy arrays con la posición del gato y el ratón ``(cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1])``.

Además, generaremos un set de etiquetas ``y`` que contendrá numpy arrays con el movimiento a realizar (ej. ``(1, 0, 0, 0, 0)`` significa hacer el movimiento 0).

La forma en que calculas qué acción llevar a cabo dado el estado de juego es tu decisión. Lo importante es que este será el comportamiento que la red neuronal del agente tratará de aproximar. Recuerda que dispones del mapa de juego en ``lab_map`` y de un agente contra el que entrenar bajo la clase `BaseCat`.

In [38]:
# Instanciamos al jugador oponente
rival = BaseCat((0, 0))

X = []
y = []

for cat_pos in free_positions:
    for mouse_pos in free_positions:
        X.append(np.array([cat_pos[0], cat_pos[1], mouse_pos[0], mouse_pos[1]]))

        # ===== COMPLETAR =====
        # Se debe calcular un vector de accion, correspondiente a la decisión a tomar para añadirlo a la lista y
        # label = np.zeros((5))   # Actualmente se trata de un vector de solo ceros, se debe reemplazar la posición del movimiento elegido por un 1
        # =====================

        # Calculamos la acción que tomaría el gato
        cat_action = rival.get_action(lab_map, cat_pos, mouse_pos)

        # Representamos la acción en formato one-hot
        label = np.zeros((5))
        label[cat_action] = 1

        y.append(label)

In [45]:
print(len(X), len(y))

6561 6561


In [39]:
# Transformamos las listas a numpy arrays
X = np.array(X)
y = np.array(y)

### Diseño y entrenamiento de la Red Neuronal

Para entrenar nuestra red neuronal, utilizaremos la librería de [`TensorFlow`](https://www.tensorflow.org/api_docs/python/tf/all_symbols), podemos instalar esta librería mediante la línea `pip install tensorflow`.

TensorFlow es una librería desarrollada por Google que nos permite construir, entrenar e implementar modelos de aprendizaje profundo en Python.

Para esto, TensorFlow nos permite armar una red en forma de "capas", [en el siguiente link encontrarás un tutorial más en detalle de cómo crear un modelo de TensorFlow](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5). Deberemos introducir todas las capas de nuestra red dentro de un objeto `tf.keras.Sequential()`, que recibe de parámetro una lista con todos los elementos de nuestra red.

<br>

A continuación se enumeran los tres principales elementos que utilizaremos en nuestro modelo:
- `tf.keras.Input(shape = a)`: La capa de entrada de la red, es la primera que recibe. Necesita de un parámetro `shape` que determinará la dimensionalidad del vector de entrada, esto será entregado en una tupla `a` (ej. `(5, 2)`).

- `tf.keras.layers.Dense(units = b, activation = 'relu')`: Corresponde a una capa intermedia de la red del tipo *Fully Connected*, le entregaremos un parámetro `units` que determinará cuántas unidades ocultas (neuronas) tendrá la capa (y por tanto, el número de componentes en el vector de salida de la capa, en este caso `b`). Además, recibirá una función de activación para cada neurona, esta puede ser del tipo `relu`, `tanh`, `softmax` u otras, para más detalle sobre ellas visitar [el siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/activations). Recomendamos usar `relu` para capas intermedias.

- `tf.keras.layers.Dense(units = c, activation = 'softmax')`: Similar a lo explicado anteriormente, corresponde a una capa densa de activación `softmax`. Esta corresponderá a la capa final del modelo, con `c` el número de elementos que deseamos que nuestro vector de salida tenga. El uso de una función softmax se debe a que esta nos permite llevar la salida de nuestra neurona a un conjunto de probabilidades normalizadas, desde el cual podemos calcular qué acción jugar. [En el siguiente link](https://deepai.org/machine-learning-glossary-and-terms/softmax-layer) se explica el funcionamiento de esta función en mayor detalle.

<br>

Un ejemplo de una red simple sería:
```
model = tf.keras.Sequential([
    tf.keras.Input(shape = a),
    tf.keras.layers.Dense(b, activation = "relu"),
    tf.keras.layers.Dense(c, activation = "softmax")
])
```

In [40]:
# Recuerda que para esta tarea la entrada es un vector de 4 elementos y buscamos una salida de 5 elementos (la probabilidad de jugar acción posible)

# ===== COMPLETAR =====
# Se debe crear un modelo para nuestra red neuronal, añade cuantas capas ocultas desees, con cuantas neuronas desees
# model = tf.keras.Sequential([])
# =====================

model = tf.keras.Sequential([
    # Capa de entrada
    tf.keras.Input(shape=(4,)),  # La entrada es un vector de 4 elementos

    # Capas ocultas
    # tf.keras.layers.Dense(32, activation="relu"),  # Primera capa oculta con 32 neuronas
    # tf.keras.layers.Dense(64, activation="relu"),  # Segunda capa oculta con 64 neuronas
    # tf.keras.layers.Dense(128, activation="relu"),  # Tercera capa oculta con 128 neuronas 
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(500, activation='relu'),

    # Capa de salida
    tf.keras.layers.Dense(5, activation="softmax")  # La salida es un vector de 5 elementos (probabilidad para cada acción)
])

TensorFlow nos permite además compilar un modelo, especificando qué optimizador utilizar para su ajuste de parámetros y qué función de pérdida utilizar para hacer la backpropagation.

Compilaremos un modelo de la forma:
``model.compile(optimizer = 'optimizer_name', loss = 'loss_name', metrics = ['accuracy'])``

- Con `optimizer_name` el nombre de algún optimizador (por ejemplo, `sgd` o `adam`, más optimizadores en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers))
- Con `loss_name` el nombre de alguna función de pérdida (por ejemplo, `binary_crossentropy`, más funciones en el [siguiente link](https://www.tensorflow.org/api_docs/python/tf/keras/losses))
- Usaremos de métrica el accuracy del modelo, aunque existe [múltiples otras métricas que incluir](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)

In [41]:
# ===== COMPLETAR =====
# Compila tu modelo de TensorFlow
# model
# =====================

model.compile(
    optimizer='adam',  # Optimizador
    loss='categorical_crossentropy',  # Función de pérdida
    metrics=['accuracy']  # Métricas para monitorear durante el entrenamiento
)

Ahora que especificamos cómo vamos calcular el error del modelo y cómo lo usaremos para ajustar sus pesos, es hora de entrenar nuestra red. Para ello, utilizaremos su método `.fit()` de la siguiente forma:

`model.fit(X, y, epochs = a, batch_size = b)`

- `X` corresponde a nuestra matriz de características
- `y` corresponde a nuestro vector de etiquetas para cada muestra
- `epochs` determina cuántas veces iteraremos sobre todo el set de datos
- `batch_size` determina cuántas muestras serán consideradas por propagación del gradiente (un valor mayor significa entrenamiento más rápido pero mayor uso de memoria)

In [52]:
# ===== COMPLETAR =====
# Entrena tu modelo de TensorFlow utilizando su método .fit()
# model
# =====================

# Entrenamiento del modelo
history = model.fit(
    X,  # Matriz de características
    y,  # Vector de etiquetas
    epochs=20,  # Número de épocas
    batch_size=256  # Tamaño del lote
)


Epoch 1/20
26/26 [==============================] - 0s 7ms/step - loss: 0.0662 - accuracy: 0.9761
Epoch 2/20
26/26 [==============================] - 0s 6ms/step - loss: 0.0712 - accuracy: 0.9733
Epoch 3/20
26/26 [==============================] - 0s 8ms/step - loss: 0.0732 - accuracy: 0.9720
Epoch 4/20
26/26 [==============================] - 0s 6ms/step - loss: 0.0748 - accuracy: 0.9736
Epoch 5/20
26/26 [==============================] - 0s 5ms/step - loss: 0.1024 - accuracy: 0.9596
Epoch 6/20
26/26 [==============================] - 0s 5ms/step - loss: 0.0869 - accuracy: 0.9672
Epoch 7/20
26/26 [==============================] - 0s 5ms/step - loss: 0.0672 - accuracy: 0.9779
Epoch 8/20
26/26 [==============================] - 0s 5ms/step - loss: 0.0634 - accuracy: 0.9774
Epoch 9/20
26/26 [==============================] - 0s 6ms/step - loss: 0.0639 - accuracy: 0.9767
Epoch 10/20
26/26 [==============================] - 0s 5ms/step - loss: 0.0607 - accuracy: 0.9785
Epoch 11/20
26/26 [

Por último, guardaremos el modelo entrenado en el directorio `agents/data/NNMouse.h5` para poder acceder a este modelo desde nuestra clase `NNMouse`:

In [110]:
model.save(os.path.join('agents', 'data', 'NNMouse.h5'))

## Todas las respuestas a las preguntas se encuentran en el pdf IA_T4.pdf